Complaint problems:
Date range: 2014-2015
Only closed complaints (exclude “conditions are still open” complaints)

1. Read in housing maintenance code complaints dataset (HMCC)(only records with status dates after 2014, and statusid = 2), including only the following features:
-complaintID
-boroughID
-block
-lot
2. Construct the BBL using the Borough, block, and lot features in the housing 

In [10]:
import pandas as pd
import numpy as np

#to use API calls
def get_data():
    '''
    Returns DataFrame with housing maintenance code complaints dataset (only records with status dates after 2014, 
    and statusid = 2)
    '''
    query = ("https://data.cityofnewyork.us/resource/jspd-2hr7.json?statusid=2"
             "&$where=statusdate%20between%20'2014-01-01T00:00:00'%20and%20'2015-10-01T00:00:00'"
             "&$limit=1000")
    hmc_complaints = pd.read_json(query)
    i = 1 
    more_data_available = True
    while more_data_available:
        query = ("https://data.cityofnewyork.us/resource/jspd-2hr7.json?statusid=2"
                 "&$where=statusdate%20between%20'2014-01-01T00:00:00'%20and%20'2015-10-01T00:00:00'"
                 "&$limit=1000&$order=:id&$offset=" + str(i*1000))
        more_data = pd.read_json(query)
        hmc_complaints = pd.concat([hmc_complaints, more_data],ignore_index=True)
        print 'Currently have ', len(hmc_complaints), ' records'
        i += 1
        
        if len(more_data) < 1000:
            more_data_available = False
    return pd.DataFrame(hmc_complaints)


#API approach
data = get_data()
print data.shape

In [11]:
#Standart approach

data = pd.read_csv("HMCC.csv")

In [12]:
def make_BBL(borough, block, lot): 
    '''
    The borough code is one numeric digit. 
    The tax block is one to five numeric digits, preceded with leading zeros 
    when the block is less than five digits.
    The tax lot is one to four digits and is preceded with leading zeros
    when the lot is less than four digits.
    
    >>> make_BBL(1,16,100)
    1000160100
    >>> make_BBL(3,15828,7501)
    3158287501
    '''
    return int(str(borough) + str(block).zfill(5) + str(lot).zfill(4))
    
data = data[['ComplaintID', 'BoroughID', 'Block', 'Lot']]
data['BBL'] = map(make_BBL, data['BoroughID'], data['Block'], data['Lot'])
print data.head(n = 5)

   ComplaintID  BoroughID  Block  Lot         BBL
0      6960137          1   1904    4  1019040004
1      6960832          1   1918    4  1019180004
2      6946867          1   1083    1  1010830001
3      6966946          1   2073   29  1020730029
4      6963755          1   1869   44  1018690044
